# 波士頓房價預測

In [1]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense

np.random.seed(7)

D:\Python\envs\keras\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
df = pd.read_csv("./boston_housing.csv")
dataset = df.values
np.random.shuffle(dataset)

## 分割特徵標籤資料及建立標準化

In [3]:
X = dataset[:, 0:13]
Y = dataset[:, 13]
X -= X.mean(axis=0)
X /= X.std(axis=0)
X_train,Y_train = X[:404],Y[:404]
X_test,Y_test = X[404:],Y[404:]

## 建立模型

In [4]:
def build_model():
    model = Sequential()
    model.add(Dense(32,input_shape=(X_train.shape[1],),activation="relu"))
    model.add(Dense(1))
    model.compile(loss="mse",optimizer="adam",metrics="mae")
    return model

## K-fold交叉驗證

In [5]:
k = 4
nb_val_samples = len(X_train) // k
nb_epochs = 80
mse_scores = []
mae_scores = []

In [6]:
for i in range(k):
    print("Processing Fold #" + str(i))
    X_val = X_train[i*nb_val_samples: (i+1)*nb_val_samples]
    Y_val = Y_train[i*nb_val_samples: (i+1)*nb_val_samples]
    X_train_p = np.concatenate(
    [X_train[:i*nb_val_samples],X_train[(i+1)*nb_val_samples:]],axis=0)
    Y_train_p = np.concatenate(
    [Y_train[:i*nb_val_samples],Y_train[(i+1)*nb_val_samples:]],axis=0)

Processing Fold #0
Processing Fold #1
Processing Fold #2
Processing Fold #3


## 訓練模型

In [7]:
model = build_model()
model.fit(X_train_p,Y_train_p,epochs=nb_epochs,batch_size=16,verbose=0)
mse,mae = model.evaluate(X_val,Y_val)
mse_scores.append(mse)
mae_scores.append(mae)

4/4 [==============================] - 0s 4ms/step - loss: 23.1178 - mae: 3.5973


In [8]:
print("MSE_val",np.mean(mse_scores))
print("MAE_val",np.mean(mae_scores))
mse,mae = model.evaluate(X_test,Y_test)
print("X_test",mse)
print("Y_test",mae)

MSE_val 23.117753982543945
MAE_val 3.5972633361816406
4/4 [==============================] - 0s 3ms/step - loss: 16.2198 - mae: 3.2532
X_test 16.219785690307617
Y_test 3.2531981468200684


### 結語: 房價誤差3253美元